In [ ]:
#r "nuget: FSharp.Data, 4.2.7"
#r "nuget: Deedle, 2.5.0"
#r "nuget: FSharp.Stats, 0.4.3"
#r "nuget: Cyjs.NET, 0.0.4"


[![Binder](https://fslab.org/images/badge-binder.svg)](https://mybinder.org/v2/gh/fslaborg/fslaborg.github.io/gh-pages?filepath=content/tutorials/007_replicate-quality-control.ipynb)&emsp;
[![Script](https://fslab.org/images/badge-script.svg)](https://fslab.org/content/tutorials/007_replicate-quality-control.fsx)&emsp;
[![Notebook](https://fslab.org/images/badge-notebook.svg)](https://fslab.org/content/tutorials/007_replicate-quality-control.ipynb)

# Replicate quality control


_Summary:_ This tutorial demonstrates an example workflow using different FsLab libraries. The aim is to check the quality of replicate measurements by clustering the samples.


## Introduction

In biology and other sciences, experimental procedures are often repeated several times in the same conditions. These resulting samples are called replicates. 
Replicates are especially useful to check for the reproducibility of the results and to boost their trustability.

One metric for the quality of the measurements is rather easy in principle. Samples received from a similar procedure should also result in similar measurements. 
Therefore just checking if replicates are more similar than other samples can already hand to the experimenter some implications about the quality of his samples.
This is especially useful when considering that usually - as the ground truth is unknown - this trustability is difficult to measure. 

In this tutorial, a simple workflow will be presented for how to visualize the clustering of replicates in an experiment. For this, 3 FsLab libraries will be used:

0. [FSharp.Data](https://fsprojects.github.io/FSharp.Data/) for retreiving the data file
1. [Deedle](https://github.com/fslaborg/Deedle) for reading a frame containing the data
2. & 3. [FSharp.Stats](https://fslab.org/FSharp.Stats/) to impute missing values and cluster the samples
4. [CyJS.NET](https://fslab.org/Cyjs.NET/) to visualize the results


## Referencing packages

```fsharp
#r "nuget: FSharp.Data"
#r "nuget: Deedle"
#r "nuget: FSharp.Stats"
#r "nuget: Cyjs.NET"

do fsi.AddPrinter(fun (printer:Deedle.Internal.IFsiFormattable) -> "\n" + (printer.Format()))
```

## Loading Data 

In this tutorial, an in silico generated dataset is used.  

`FSharp.Data` and `Deedle` are used to load the data into the fsi.




In [3]:
open FSharp.Data
open Deedle

// Load the data 
let rawData = Http.RequestString @"https://raw.githubusercontent.com/fslaborg/datasets/main/data/InSilicoGeneExpression.csv"

// Create a deedle frame and index the rows with the values of the "Key" column.
let rawFrame : Frame<string,string> = 
    Frame.ReadCsvString(rawData)
    |> Frame.indexRows "Key"


Condition0_1     Condition0_2     Condition0_3     Condition1_1     Condition1_2     Condition1_3     Condition2_1     Condition2_2     Condition2_3     Gene0  -> <missing>        <missing>        859.507048737706 892.488061131967 1018.39682842723 <missing>        1103.47465251202 1157.72940330711 1065.74060396554 Gene1  -> 874.831680800388 750.248739657293 885.186911420285 928.994516057073 853.081858812674 793.574297701139 1065.97949919587 1131.14376992316 <missing>        Gene2  -> 838.556912459832 852.727407339623 899.295260312015 860.880771705626 932.199854945633 976.124808642915 1207.93463145272 <missing>        1277.61049813247 Gene3  -> 578.81785907921  678.347549342628 602.246497320338 <missing>        643.093516693419 <missing>        <missing>        873.194740469258 849.451122811244 Gene4  -> 842.094396445274 965.835426665507 867.369051645365 928.252271146921 881.501122913359 <missing>        1054.1287942036  1171.60939846118 1038.00577431047 Gene5  -> 1020.09691148753 1074.

## Data imputation

Missing data is a constant companion of many data scientists. And it's not the best company, as missing values [can introduce a substantial amount of bias, make the handling and analysis of the data more arduous, and create reductions in efficiency](https://en.wikipedia.org/wiki/Imputation_(statistics)).

To tackle this, missing values can be substituted in a step called `imputation`. Different approaches for this exist. Here a k-nearest neighbour imputation is shown, which works as follows: 
For each observation with missing values, the k most similar other observations are chosen. Then the missing value of this observation is substituted by the mean of these values in the neighbouring observations.




In [5]:
open FSharp.Stats
open FSharp.Stats.ML

// Select the imputation method: kNearestImpute where the 2 nearest observations are considered
let kn : Impute.MatrixBaseImputation<float[],float> = Impute.kNearestImpute 2

// Impute the missing values using the "imputeBy" function. The values of the deedle frame are first transformed into the input type of this function.
let imputedData = 
    rawFrame 
    |> Frame.toJaggedArray 
    |> Impute.imputeBy kn Ops.isNan

// Creating a new frame from the old keys and the new imputed data
let imputedFrame = 
    Frame.ofJaggedArray imputedData
    |> Frame.indexRowsWith rawFrame.RowKeys
    |> Frame.indexColsWith rawFrame.ColumnKeys


Condition0_1      Condition0_2     Condition0_3      Condition1_1     Condition1_2       Condition1_3       Condition2_1     Condition2_2       Condition2_3      Gene0  -> 815.0863716692485 834.177804837712 859.507048737706  892.488061131967 1018.39682842723   993.467661383195   1103.47465251202 1157.72940330711   1065.74060396554  Gene1  -> 874.831680800388  750.248739657293 885.186911420285  928.994516057073 853.081858812674   793.574297701139   1065.97949919587 1131.14376992316   1097.76308399039  Gene2  -> 838.556912459832  852.727407339623 899.295260312015  860.880771705626 932.199854945633   976.124808642915   1207.93463145272 1106.54977345808   1277.61049813247  Gene3  -> 578.81785907921   678.347549342628 602.246497320338  650.900998152141 643.093516693419   662.8620463286804  871.917379913417 873.194740469258   849.451122811244  Gene4  -> 842.094396445274  965.835426665507 867.369051645365  928.252271146921 881.501122913359   1009.0399679729114 1054.1287942036  1171.6093984611

## Hierarchical clustering

To sort the level of closeness between samples, we perform a hierarchical clustering. Details about this can be found [here](003_clustering_hierarchical.html) and [here](https://fslab.org/FSharp.Stats/Clustering.html#Hierarchical-clustering).




In [7]:
open FSharp.Stats.ML.Unsupervised

// Retreive the sample columns from the frame
let samples = 
    imputedFrame
    |> Frame.getNumericCols
    |> Series.observations
    |> Seq.map (fun (k,vs) -> 
        k,
        vs
        |> Series.values
    )

// Run the hierarchical clustering on the samples
// The clustering is performed on labeled samples (name,values) so that these labels later appear in the cluster tree
let clustering = 
    HierarchicalClustering.generate 
        (fun (name1,values1) (name2,values2) -> DistanceMetrics.euclidean values1 values2) // perform the distance calculation only on the values, not the labels
        HierarchicalClustering.Linker.wardLwLinker
        samples
    |> HierarchicalClustering.mapClusterLeaftags fst // only keep the labels in the cluster tree


Node  (16, 7262.367644, 9,   Node     (15, 1317.505683, 6,      Node        (14, 765.0062545, 3,         Node           (13, 750.1592066, 2, Leaf (3, 1, "Condition1_1"),            Leaf (5, 1, "Condition1_3")), Leaf (4, 1, "Condition1_2")),      Node        (11, 744.3808016, 3,         Node           (9, 684.1625739, 2, Leaf (0, 1, "Condition0_1"),            Leaf (1, 1, "Condition0_2")), Leaf (2, 1, "Condition0_3"))),   Node     (12, 745.2649938, 3,      Node        (10, 701.3010992, 2, Leaf (6, 1, "Condition2_1"),         Leaf (7, 1, "Condition2_2")), Leaf (8, 1, "Condition2_3")))

## Data visualization

Finally, the clustering results can be visualized to check for replicate clustering. For this we use `Cyjs.NET`, an FsLab library which makes use of the `Cytoscape.js` network visualization tool.

Further information about styling the graphs can be found [here](https://fslab.org/Cyjs.NET/).



In [9]:
open Cyjs.NET

// Function for flattening the cluster tree to an edgelist
let hClustToEdgeList (f : int -> 'T) (hClust : HierarchicalClustering.Cluster<'T>) =
    let rec loop (d,nodeLabel) cluster=
        match cluster with
        | HierarchicalClustering.Node (id,dist,_,c1,c2) ->
            let t = f id
            loop (dist,t) c1
            |> List.append (loop (dist,t) c2)
            |> List.append [nodeLabel,t,d] 
        | HierarchicalClustering.Leaf (_,_,label)-> [(nodeLabel,label,d)]
    loop (0., f 0) hClust

let rawEdgeList = hClustToEdgeList (string) clustering

// The styled vertices, samnples are coloured based on the condition they belong to. So replicates of one condition have the same colour
let cytoVertices = 
    rawEdgeList
    |> List.collect (fun (v1,v2,w) ->
        [v1;v2]
    )
    |> List.distinct
    |> List.map (fun v -> 
        let label,color,size = 
            match v.Split '_' with
            | [|"Condition0";_|] -> "Condition0", "#6FB1FC","40"
            | [|"Condition1";_|] -> "Condition1", "#EDA1ED","40"
            | [|"Condition2";_|] -> "Condition2", "#F5A45D","40"
            | _ -> "","#DDDDDD","10"

        let styling = [CyParam.label label; CyParam.color color; CyParam.width size]
        Elements.node (v) styling
    )

// Helper function to transform the distances between samples to weights
let distanceToWeight = 
    let max = rawEdgeList |> List.map (fun (a,b,c) -> c) |> List.max
    fun distance -> 1. - (distance / max)   


// Styled edges
let cytoEdges = 
    rawEdgeList
    |> List.mapi (fun i (v1,v2,weight) -> 
        let styling = [CyParam.weight (distanceToWeight weight)]
        Elements.edge ("e" + string i) v1 v2 styling
    )

// Resulting cytograph
let cytoGraph = 

    CyGraph.initEmpty ()
    |> CyGraph.withElements cytoVertices
    |> CyGraph.withElements cytoEdges
    |> CyGraph.withStyle "node" 
        [
            CyParam.content =. CyParam.label
            CyParam.shape =. CyParam.shape
            CyParam.color =. CyParam.color
            CyParam.width =. CyParam.width
        ]
    |> CyGraph.withLayout (Layout.initCose (id))  


```fsharp
// Send the cytograph to the browser
cytoGraph
|> CyGraph.show
```


```
No value returned by any evaluator
```

## Interpretation

As can be seen in the graph, replicates of one condition cluster together. This is a good sign for the quality of the experiment. 
If one replicate of a condition does not behave this way, it can be considered an outlier.
If the replicates don't cluster together at all, there might be some problems with the experiment.


